In [ ]:
pip install requests keyboard twilio


In [1]:
import requests
from time import sleep
from atmiv import atm_iv  # Ensure atmiv module is correctly installed
from twilio.rest import Client
import keyboard  # For detecting key press
import math 


# Twilio setup (replace with your Twilio credentials)
account_sid = '
auth_token = ''
twilio_whatsapp_number = '6'  # This is the Twilio sandbox WhatsApp number
your_whatsapp_number = ''

client = Client(account_sid, auth_token)

date = "270824"

# Define the symbols and URL
call_btc = "C-BTC-63400-270824"
put_btc = "P-BTC-63600-270824"
call_eth = "C-ETH-2720-270824"
put_eth = "P-ETH-2720-270824"

# Buy prices
Buy_call_btc_Price = 644
Buy_put_btc_Price = 533
Buy_call_eth_Price = 31.42
Buy_put_eth_Price = 26.84

buy_total = Buy_call_btc_Price + Buy_put_btc_Price + Buy_call_eth_Price + Buy_put_eth_Price

def atm_iv(date):
    def find_atm(symbol,gap):
        

        response = requests.get("https://cdn.india.deltaex.org/v2/tickers" + f"/{symbol}")
        ticker_info = response.json()
        btc_price = ticker_info['result']['close']
        atm_value =  int(math.floor(btc_price / gap)) * gap
        return atm_value
        
 

    call_btc_atm = f"C-BTC-{find_atm("BTCUSD",200)}-{date}"
    put_btc_atm = f"P-BTC-{find_atm("BTCUSD",200)}-{date}"
    call_eth_atm = f"C-ETH-{find_atm("ETHUSD",20)}-{date}"
    put_eth_atm = f"P-ETH-{find_atm("ETHUSD",20)}-{date}"

    def get_iv(symbol):
            url = f"https://cdn.india.deltaex.org/v2/tickers/{symbol}"
            response = requests.get(url)
            ticker_info = response.json()
            mark_iv = float(ticker_info['result']['quotes']['mark_iv'])
            return mark_iv*100 


    btc_call_iv = get_iv(call_btc_atm)
    btc_put_iv = get_iv(put_btc_atm)
    btc_total_iv = (btc_put_iv + btc_call_iv)/2
    eth_call_iv = get_iv(call_eth_atm)
    eth_put_iv = get_iv(put_eth_atm)
    eth_total_iv = (eth_put_iv + eth_call_iv)/2
    print(f"BTC:ATM_IV {btc_total_iv}% Expiry Date:{date}")  
    print(f"ETH:ATM_IV {eth_total_iv}% Expiry Date:{date}")
        

def get_price(symbol):
    try:
        url = f"https://cdn.india.deltaex.org/v2/tickers/{symbol}"
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad responses
        ticker_info = response.json()
        mark_price_str = ticker_info['result']['mark_price']
        return float(mark_price_str)
    except Exception as e:
        print(f"Error fetching price for {symbol}: {e}")
        return None

def send_whatsapp_alert(message):
    try:
        message = client.messages.create(
            body=message,
            from_=twilio_whatsapp_number,
            to=your_whatsapp_number
        )
        print(f"Alert sent: {message.sid}")
    except Exception as e:
        print(f"Failed to send alert: {e}")

def fetch_data():
    # Fetch prices
    current_btc_call_price = get_price(call_btc)
    current_btc_put_price = get_price(put_btc)
    current_eth_call_price = get_price(call_eth)
    current_eth_put_price = get_price(put_eth)  
    current_total = current_btc_call_price + current_btc_put_price + current_eth_call_price + current_eth_put_price
    per_change = ((current_total - buy_total) / buy_total) * 100
   
    print(f"Percentage Change: {per_change:.2f}%")

    # Send alert if percentage change is above 50%
    if per_change > 0:
        send_whatsapp_alert(f"Alert! Percentage change is above 0%. Current change: {per_change:.2f}%")

# Start fetching data continuosusly
while True:
    fetch_data()
    atm_iv(date)
    sleep(30)

    # Check if 's' is pressed to stop the loop
    if keyboard.is_pressed('s'):
        print("Stopping the loop.")
        break


Percentage Change: -2.79%
BTC:ATM_IV 40.87789% Expiry Date:270824
ETH:ATM_IV 49.0908065% Expiry Date:270824
Percentage Change: -2.86%
BTC:ATM_IV 40.9265515% Expiry Date:270824
ETH:ATM_IV 49.2655945% Expiry Date:270824
Percentage Change: -3.08%
BTC:ATM_IV 40.899227499999995% Expiry Date:270824
ETH:ATM_IV 49.248225500000004% Expiry Date:270824
Percentage Change: -2.75%
BTC:ATM_IV 41.0344865% Expiry Date:270824
ETH:ATM_IV 49.482406% Expiry Date:270824
Percentage Change: -3.20%
BTC:ATM_IV 41.0158165% Expiry Date:270824
ETH:ATM_IV 49.467005% Expiry Date:270824
Percentage Change: -3.59%
BTC:ATM_IV 40.964541000000004% Expiry Date:270824
ETH:ATM_IV 49.506648999999996% Expiry Date:270824
Percentage Change: -3.85%
BTC:ATM_IV 40.905525% Expiry Date:270824
ETH:ATM_IV 49.463277% Expiry Date:270824
Percentage Change: -3.80%
BTC:ATM_IV 40.876632% Expiry Date:270824
ETH:ATM_IV 49.372437500000004% Expiry Date:270824
Percentage Change: -3.83%
BTC:ATM_IV 40.882709500000004% Expiry Date:270824
ETH:ATM_IV 

KeyboardInterrupt: 